# Chapter 1: the Python Data Model
## 1.1 一摞Python风格的纸牌

In [ ]:
import collections
Card=collections.namedtuple('Card',['rank','suit']) # namedtuple常用以构建只有一些属性(attributes)但没有方法(methods)的类(class)
class FrenchDeck:
    ranks=[str(n) for n in range(2,11)]+list('JQKA')
    suits='spades diamonds clubs hearts'.split()
    def __init__(self):
        self._cards=[Card(rank,suit) for suit in self.suits for rank in self.ranks]
    def __len__(self):
        return len(self._cards)
    def __getitem__(self,position):
        return self._cards[position]


In [ ]:
beer_card=Card('7','diamonds')
beer_card  # Card(rank='7', suit='diamonds')
deck=FrenchDeck()
len(deck) # 52 ;或者deck.__len__()
deck[0] # Card(rank='2', suit='spades')
deck[-1] # Card(rank='A', suit='hearts')
from random import choice
choice(deck)
# 因为__getitem__方法把[]操作交给了self._cards列表，因此，deck类自动支持切片
deck[:3]
deck[12::13] # 先抽出索引号是12的牌，然后每13张取一张牌
for card in deck: #  for card in reversed(deck): # 反向迭代
       print(card)
#当一个集合类型没有实现__contains__方法时，in运算就会按顺序做一次迭代搜索（前提是可迭代的）
Card("Q","hearts") in deck # True
# 排序，依据是点数以及花色：黑桃最大，红桃次之，方块再次，梅花最小;可以通过定义一个spades_high函数来作为sorted的key
suit_values=dict(spades=3,hearts=2,diamonds=1,clubs=0)
def spades_high(card):
    rank_value=FrenchDeck.ranks.index(card.rank) # 返回list元素的index
    return rank_value*len(suit_values)+suit_values[card.suit]
for card in sorted(deck,key=spades_high):
    print(card)

## 1.2 如何使用特殊方法

In [ ]:
# 自定义一个二维向量，可以实现向量的模(hypot)，向量加法以及向量与标量的乘法等，同时结果能被友好地打印出来(__repr__)
from math import hypot
class Vector:
    def __init__(self,x=0,y=0):
        self.x=x
        self.y=y
    def __repr__(self):
        return 'Vector(%r,%r)'%(self.x,self.y)
    def __abs__(self):
        return hypot(self.x,self.y)
    def __bool__(self):
        return bool(abs(self))
    def __add__(self,other):
        x=self.x+other.x
        y=self.y+other.y
        return Vector(x,y)
    def __mul__(self,scalar):
        return Vector(self.x*scalar,self.y*scalar)
print(Vector(1,2))

## 1.3 特殊方法一览

## 1.4 为什么len是特殊方法，而不是普通方法
* 其他方法也是同理：len之所以不是一个普通方法，是为了让Python自带的数据结构可以走后门(直接使用CPython)

# Chapter 2: 序列构成的数组
## 2.1 内置序列类型概览

In [ ]:
# python内置的序列主要分为两种类型：
## 容器序列(允许不同元素)，例如：list、tuple、collections.deque；容器序列存放的是任意类型的引用
## 另外一种是扁平序列(只能存放一种数据类型)，例如：str、bytes、bytearray、memoryview和array.array；扁平序列存放的是值而不是引用，即扁平序列其实是一段连续的内存空间


In [ ]:
# 如果将序列按照能否被修改来分类，则可以分为
## 可变序列MutableSequence：list、bytearray、array.array、collections.deque和memoryview
## 不可变序列Sequence：tuple、str和bytes

## 2.2 列表推导和生成器表达式
* 通常的原则是只用列表推导式来创新新的列表，并且尽量保持简短（注意Python会忽略代码里[]、{}、()中的换行）

In [ ]:
symbols='abcdefg'
codes=[]
for symbol in symbols:
    codes.append(ord(symbol))
codes

In [ ]:
# 列表推导同filter和map的比较
symbols='abcdefg'
beyond_ascii=[ord(s) for s in symbols if ord(s)>100]
beyond_ascii
beyond_ascii=list(filter(lambda c:c>100,map(ord,symbols)))
beyond_ascii
# 注意 map/filter 组合起来用不一定比列表推导式快
import timeit

TIMES = 10000

SETUP = """
symbols = '$¢£¥€¤'
def non_ascii(c):
    return c > 127
"""

def clock(label, cmd):
    res = timeit.repeat(cmd, setup=SETUP, number=TIMES)
    print(label, *('{:.3f}'.format(x) for x in res))

clock('listcomp        :', '[ord(s) for s in symbols if ord(s) > 127]')
clock('listcomp + func :', '[ord(s) for s in symbols if non_ascii(ord(s))]')
clock('filter + lambda :', 'list(filter(lambda c: c > 127, map(ord, symbols)))')
clock('filter + func   :', 'list(filter(non_ascii, map(ord, symbols)))')

In [ ]:
# 生成器(可以减少内存消耗)，语法与列表推导式相似，只是把方括号换成圆括号
## 注意，如果生成器是某函数的唯一参数，则不需要圆括号，例如
symbols='abcdefghijklmn'
tuple(ord(symbol) for symbol in symbols)

## 2.3 元组tuple不仅仅是不可变的列表
* 元组实际上是对数据内容和未知的记录

In [ ]:
# 元组拆包
a=('Harbes','Arvin')
b,c=a
b
b,c=c,b
c
b,c

In [ ]:
# 使用 * 运算符把一个可迭代对象超开胃函数的参数
divmod(20,8)
paras=(20,8)
divmod(*paras)

a="I'm Harbes"
_,b=a.split() # 拆包时，不是对所有元素都感兴趣
b

a="I'm Harbes Arvin"
*_,b=a.split()
b
_,b # (["I'm", 'Harbes'], 'Arvin')

a,b,*rest=range(3)
a,b,rest # (0, 1, [2])
a,b,*rest=range(2)
a,b,rest # (0, 1, [])

In [ ]:
# 嵌套元组拆包
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),   # <1>
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

print('{:15} | {:^9} | {:^9}'.format('', 'lat.', 'long.'))
fmt = '{:15} | {:9.4f} | {:9.4f}'
for name, cc, pop, (latitude, longitude) in metro_areas:  # <2>
    if longitude <= 0:  # <3>
        print(fmt.format(name, latitude, longitude))

In [ ]:
# namedtuple (from collections import namedtuple) 用来构建一个带字段名的元组和一个有名字的类
from collections import namedtuple
City=namedtuple('city','name country population coordinates') # City是类，city是名
tokyo=City('Tokyo','JP',36,(36,140))
tokyo # city(name='Tokyo', country='JP', population=36, coordinates=(36, 140))
tokyo.population
# 除了从普通元组那里继承来的属性之外，namedtuple还有一些自己专有的属性，例如 _fileds类属性, 类方法_make(iterable)和实例方法 _asdict()
City._fields # ('name', 'country', 'population', 'coordinates') # 返回包含这个类所有字段名称的元组
City._asdict(tokyo) # OrderedDict([('name', 'Tokyo'),('country', 'JP'),('population', 36),('coordinates', (36, 140))])
t=City._make(('Tokyo','JP',36,(36,140))) # city(name='Tokyo', country='JP', population=36, coordinates=(36, 140)) # _make() 通过接受一个可迭代对象来生成这个类的一个实例
City._asdict(t) # OrderedDict([('name', 'Tokyo'),('country', 'JP'),('population', 36),('coordinates', (36, 140))])
for key,value in tokyo._asdict().items():
    print(key+':',value)

In [ ]:
# 作为不可变列表的元组（列表与元组的异同）

## 2.4 切片

In [ ]:
# 为什么切片以及区间会忽略最后一个元素，例如：a[:3]不返回a[3]
### 当只有最后一个位置信息时，可以快速看出切片和区间的元素个数，例如a[:3]就是3个元素
### 当起止信息可见时，，可以快速计算切片和区间的长度：stop-start
### 可以利用任意一个下标将序列分割成不重叠的两部分：a[:x]和a[x:]

In [ ]:
# s[a:b:c]的形式对s在a和b之间以c为间隔取值；若c为负值，则反向取值

In [ ]:
# 多维切片和省略
## a[m:n,x:y]
## a[m,...] # 注意使用 ... 

In [ ]:
# 给切片赋值，注意：若要赋值的对象是一个切片，那么赋值语句右侧一定是一个可迭代对象iterable
li=list(range(10))
#li[2:5]=888 # 报错
li[2:5]=[888];li

## 2.5 对序列使用+和* 
* 一些陷阱：当a*n这个语句中，序列a中的元素是对其他可变对象的引用的话，需要引起格外的注意

In [ ]:
# 当我们使用 * 来创建一个由列表组成的列表时，要注意，列表中的所有列表实际上是指代同一个对象
board=[["_"]*3 for i in range(3)]
board[1][2]='x'
board # [['_', '_', '_'], ['_', '_', 'x'], ['_', '_', '_']]
weird_board=[['_']*3]*3
weird_board[1][2]='0'
weird_board # [['_', '_', '0'], ['_', '_', '0'], ['_', '_', '0']]

## 2.6 序列的增量赋值

In [ ]:
# *= 和 += 都有就地运算和非就地运算两种，取决于前面的对象是否是可变对象，例如
li=[1,2,3]
print(id(li)) # 2489839375432
li *=2
li # [1, 2, 3, 1, 2, 3]
print(id(li)) # 2489839375432

In [ ]:
tu=(1,2,3)
print(id(tu))
tu *=2
print(tu)
print(id(tu)) # 两次的id是不同的；由此可知，对不可变序列进行重复拼接操作的话，效率会很低，因为每次都会产生新的对象，解释器需要把原来的对象重新指向新的对象

In [ ]:
# 关于 += 的谜题
t=(1,2,[888])
t[2] +=[888] # TypeError

In [ ]:
# 上面虽然抛出了 TypeError 异常，但是print(t)的结果告诉我们，t改变了
print(t)

In [ ]:
# 除了使用try except之外，上面的命令还可以通过 使用extend来避免异常
t=(1,2,[888])
t[2].extend([888])
t

## 2.7 list.sort方法和内置函数sorted
* list.sort方法会就低排序列表，也就是说不会把原列表复制一份；而sorted会新建一个列表作为返回值。二者都有两个可选参数：reverse和key，其中key传入的是一个只有一个参数的函数。

## 2.8 用bisect来管理已排序的序列
* bisect模块包含两个主要函数：bisect和insort，都是利用二分查找算法来在有序序列中查找或插入元素。

In [ ]:
import bisect
import sys

HAYSTACK = [1, 4, 5, 6, 8, 12, 15, 20, 21, 23, 23, 26, 29, 30]
NEEDLES = [0, 1, 2, 5, 8, 10, 22, 23, 29, 30, 31]

ROW_FMT = '{0:2d} @ {1:2d}    {2}{0:<2d}'

def demo(bisect_fn):
    for needle in reversed(NEEDLES):
        position = bisect_fn(HAYSTACK, needle)  # <1>
        offset = position * '  |'  # <2>
        print(ROW_FMT.format(needle, position, offset))  # <3>

if __name__ == '__main__':

    if sys.argv[-1] == 'left':    # <4>
        bisect_fn = bisect.bisect_left
    else:
        bisect_fn = bisect.bisect

    print('DEMO:', bisect_fn.__name__)  # <5>
    print('haystack ->', ' '.join('%2d' % n for n in HAYSTACK))
    demo(bisect_fn)

In [ ]:
# 用bisect.insort插入新元素,insort可以保持有序序列的顺序
import bisect
import random

SIZE = 7

random.seed(1729)

my_list = []
for i in range(SIZE):
    new_item = random.randrange(SIZE*2)
    bisect.insort(my_list, new_item)
    print('%2d ->' % new_item, my_list)

## 2.9 当列表不是首选时

In [ ]:
# 数组,同时使用.tofile()和.fromfile()快速写入或读取文件
from array import array
from random import random 
floats=array('d',(random() for i in range(10**7)))
floats[-1]
fp=open('floats.bin','wb')
floats.tofile(fp)
fp.close()
floats2=array('d')
fp=open('floats.bin','rb')
floats2.fromfile(fp,10**7)
fp.close()
floats2[-1] # 与floats[-1]一致
# 对数组和列表的功能做了一些总结

In [ ]:
# numpy 和 scipy
### tips：使用 from time import perf_counter as pc来计时(精度和性能比较高)

In [ ]:
# 双向队列和其他形式的队列
## 双向队列是一个线程安全、可以快速从两端添加或删除元素的数据类型
from collections import deque
dq=deque(range(10),maxlen=10)
print('1 ->',dq)
dq.rotate(3) # 注意是就地变化，所以，返回的是None(print打印的结果是None)
print('2 ->',dq)
dq.rotate(-4)
print('3 ->',dq)
dq.appendleft(-1)
print('4 ->',dq)
dq.extend([11,22,33])
print('5 ->',dq)
dq.extendleft([10,20,30])
print('6 ->',dq)
# 列表和双向队列的方法

# Chapter 3: 字典和集合

In [ ]:
import this